In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re
from nltk import word_tokenize
import pandas as pd
from urllib.request import urlopen
import requests

### Import list of names from json file:

In [2]:
CharacterNames = []
with open('characters.json') as json_file:
    data = json.load(json_file)
    for p in data['characters']:
        CharacterNames.append('characterName')

### Get relevant paths

In [3]:
def GetUrls(url):

    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")
    
    AllLinks = []
    for link in soup.find_all('a'):
        AllLinks.append(link.get('href'))
    
    idx = AllLinks.index('/wiki/Season_8')
    
    AllLinks = AllLinks[idx:idx+26]
    
    indices = [AllLinks.index("/wiki/Season_" +str(i)) for i in [6,7,8]]
    for i in indices: 
        del(AllLinks[i])

    return AllLinks
    

url = "https://gameofthrones.fandom.com/wiki/"
AllUrls = GetUrls(url)
AllUrls

['/wiki/Winterfell_(episode)',
 '/wiki/A_Knight_of_the_Seven_Kingdoms',
 '/wiki/The_Long_Night',
 '/wiki/The_Last_of_the_Starks',
 '/wiki/The_Bells',
 '/wiki/The_Iron_Throne',
 '/wiki/Dragonstone_(episode)',
 '/wiki/Stormborn',
 '/wiki/The_Queen%27s_Justice',
 '/wiki/The_Spoils_of_War',
 '/wiki/Eastwatch',
 '/wiki/Beyond_the_Wall_(episode)',
 '/wiki/The_Dragon_and_the_Wolf',
 '/wiki/The_Red_Woman',
 '/wiki/Home',
 '/wiki/Oathbreaker',
 '/wiki/Book_of_the_Stranger',
 '/wiki/The_Door',
 '/wiki/Blood_of_My_Blood',
 '/wiki/The_Broken_Man',
 '/wiki/No_One',
 '/wiki/Battle_of_the_Bastards_(episode)',
 '/wiki/The_Winds_of_Winter']

### Get documents

In [4]:
def GetDocuments(root, endpath):
    url = root + endpath
    res = requests.get(url)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    text = soup.find_all(text=True)

    output = ''
    blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head', 
        'input',
        'script',
        # there may be more elements you don't want, such as "style", etc.
    ]

    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)
            
    return soup, text, output

### Accesss all captions

In [5]:
def GetSectionName(soup):
    AllH = soup.find_all(['h2', 'h3'])
    for header in AllH:
        if header.span is not None: 
            header.span.unwrap()

    AllHeaders = [header.text for header in AllH]
    Pre = AllHeaders.index('Summary')
    Post = AllHeaders.index('Appearances')

    HeadlineName = AllHeaders[Pre+1:Post]
    return HeadlineName

### Access all releveant text (including captions)

In [16]:
def GetText(HeadlineNames):
    Texts = soup.find_all(['p', 'h3'])
    for section in Texts: 
        if section is not None:
            if section.a is not None: 
                section.a.unwrap()
            elif section.span is not None: 
                section.span.unwrap()
            
    Texts = [t.text for t in Texts]
    
    CaptionIndices = []
    for caption in HeadlineNames: 
        CaptionIndices.append(Texts.index(caption))
    
    CaptionIndices.append(-1)
    
    SectionText = []
    for i in range(len(CaptionIndices)-1):
        print(i)
        SectionText.append(Texts[CaptionIndices[i]+1:CaptionIndices[i+1]])
    
    return SectionText, Texts, CaptionIndices

In [21]:
root = 'https://gameofthrones.fandom.com'
endpath = '/wiki/Winterfell_(episode)'

soup, text, output = GetDocuments(root, endpath)

HeadlineNames = GetSectionName(soup)
SectionText, Texts, CaptionIndices = GetText(HeadlineNames)
Texts[30]

0
1
2


'At Last Hearth'

In [196]:
df = pd.DataFrame(columns = ["caption", "text"])
#data = SectionText, columns = HeadlineName
df['caption'] = HeadlineName
df['text'] = SectionText
df

,caption,text
0,On Dragonstone,"[During a thunderstorm at Dragonstone, Tyrion ..."
1,In King's Landing,"[In the throne room of the Red Keep, Queen Cer..."
2,At Winterfell,"[At Winterfell, Jon Snow, Sansa Stark, and Dav..."
3,In the Riverlands,[Arya Stark encounters Hot Pie at the Inn at t...
4,In Oldtown,"[At the Citadel, Archmaester Ebrose and Samwel..."
5,On the Narrow Sea,[Yara and Ellaria's ships are attacked by Euro...
